# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Data setup 

In [2]:
X = np.load("data/X-data.npy")
y = np.load("data/y-data.npy")

In [3]:
X = X.reshape(len(X), X.shape[1], 1)

# Class definitions

In [4]:
class Layer():
    def __init__(self):
        self.input = None
        self.output = None
        
    def forward(self, input):
        pass
    
    def backward(self, output, learning_rate):
        pass

In [5]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient, self.input.T)
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return input_gradient

In [6]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [7]:
class ReLU(Activation):
    def __init__(self):
        def relu(x):
            return np.maximum(0,x)

        def relu_prime(x):
            return (x > 0) * 1

        super().__init__(relu, relu_prime)

class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))

        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)

        super().__init__(sigmoid, sigmoid_prime)

# Helper functions

In [8]:
def binary_cross_entropy(y_true, y_pred):
    return np.mean(-y_true * np.log(y_pred) - (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

In [9]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

# Training

#### Constants 

In [10]:
input_shape = X.shape[1]
learning_rate = 0.01
epochs = 100
error_viz = []

In [11]:
network = [
    Dense(input_shape, 10),
    ReLU(),
    Dense(10,1),
    Sigmoid()
]

#### Training

In [ ]:
for e in range(epochs):
    error = 0
    for x_i, y_i in zip(X, y):
        # forward
        output = predict(network, x_i)

        # error
        error += binary_cross_entropy(y_i, output)

        # backward
        grad = binary_cross_entropy_prime(y_i, output)
        for layer in reversed(network):
            grad = layer.backward(grad, learning_rate)

    error /= len(X)
    error_viz.append(error)

In [ ]:
plt.plot(error_viz)
plt.title("Training")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

# Evaluation 

In [ ]:
pred = []
for i in X:
    pred.append(predict(network, i))

In [ ]:
res = pd.DataFrame()
res["predictions"] = pred
res["actual"] = y

In [ ]:
res["predictions"] = res["predictions"].apply(lambda x: x[0][0])
res["predictions"] = res["predictions"].apply(lambda x: 0 if x < 0.5 else 1)
res

In [ ]:
print("Accuracy:",(len(res["predictions"] == res["actual"]) / len(X) * 100))